In [ ]:
!pip install roboflow
!pip install roboflow supervision opencv-python

In [ ]:
from roboflow import Roboflow
import supervision as sv
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [5]:
# Install the required package
from dotenv import load_dotenv
import os

load_dotenv()
# Get the ROBOFLOW_API key
ROBOFLOW_API_KEY = os.getenv("ROBOFLOW_API_KEY")

In [ ]:
rf = Roboflow(api_key=ROBOFLOW_API_KEY)
project = rf.workspace("imeefyp").project("swimming-pools-dctlb")
version = project.version(6)
dataset = version.download("yolov8")

In [ ]:
model = project.version(2).model

# Run inference
result = model.predict("../OpenCV_Approach/swimming_pool/000000292.jpg", confidence=40, overlap=30).json()

In [ ]:
# Read the image
image = cv2.imread("../OpenCV_Approach/swimming_pool/000000292.jpg")

In [ ]:
# Convert Roboflow predictions into `sv.Detections`
xyxy = []
class_ids = []
labels = []

In [ ]:
for pred in result["predictions"]:
    x, y, w, h = pred["x"], pred["y"], pred["width"], pred["height"]
    class_name = pred["class"]

    # Convert to [x1, y1, x2, y2] format
    x1, y1 = int(x - w / 2), int(y - h / 2)
    x2, y2 = int(x + w / 2), int(y + h / 2)

    xyxy.append([x1, y1, x2, y2])
    class_ids.append(0)  # Assign a dummy class ID (0)
    labels.append(class_name)

# Convert to supervision Detections
detections = sv.Detections(xyxy=np.array(xyxy), class_id=np.array(class_ids))

In [ ]:
# Annotators
box_annotator = sv.BoxAnnotator()
label_annotator = sv.LabelAnnotator()

# Draw Bounding Boxes
annotated_image = box_annotator.annotate(scene=image, detections=detections)
annotated_image = label_annotator.annotate(scene=annotated_image, detections=detections, labels=labels)

In [ ]:
# Display the Image in Colab using Matplotlib
plt.figure(figsize=(10, 10))
plt.imshow(cv2.cvtColor(annotated_image, cv2.COLOR_BGR2RGB))  # Convert BGR to RGB for correct display
plt.axis('off')  # Hide axes
plt.show()

In [ ]:
# Save the image
cv2.imwrite("output/detected_pools.jpg", annotated_image)